In [1]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Data/

/content/drive/MyDrive/Colab Notebooks/Data


In [4]:
!ls

animal10			 Maleria_cell_images.zip
animal10Predictor_weights.hdf5	 model.h5
archive.zip			 __pycache__
cats_and_dogs_filtered		 raw-img
cell_images			 raw-intel-images
chest_xray			 spam_email.zip
chest_xray.zip			 Tf_flower_images
flower_photos			 tf_flowers_predictor.h5
flower_photos.tar.gz		 tomato-leaf_disease
intelimage_mobilenet_weights.h5  tomato_leaf_disease_predictor.h5
intel_images.zip		 tomato_leaf.zip
malaria_mobilenet_weights.h5	 translate.py


In [5]:
!unzip spam_email.zip

Archive:  spam_email.zip
  inflating: spam_ham_dataset.csv    


In [6]:
!ls

animal10			 model.h5
animal10Predictor_weights.hdf5	 __pycache__
archive.zip			 raw-img
cats_and_dogs_filtered		 raw-intel-images
cell_images			 spam_email.zip
chest_xray			 spam_ham_dataset.csv
chest_xray.zip			 Tf_flower_images
flower_photos			 tf_flowers_predictor.h5
flower_photos.tar.gz		 tomato-leaf_disease
intelimage_mobilenet_weights.h5  tomato_leaf_disease_predictor.h5
intel_images.zip		 tomato_leaf.zip
malaria_mobilenet_weights.h5	 translate.py
Maleria_cell_images.zip


In [27]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 5.1 MB/s 


In [7]:
import pandas as pd
df = pd.read_csv('spam_ham_dataset.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [8]:
df.shape

(5171, 4)

In [9]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [43]:
df['label_num'].value_counts()

0    3672
1    1499
Name: label_num, dtype: int64

In [24]:
bert_model_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_preprocessing_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [14]:
text[1]

'Auro is a bad dog'

In [28]:
# Creating keras layers using BERT model
import tensorflow_hub as hub
import tensorflow_text as text
bert_model = hub.KerasLayer(bert_model_url)
preprocessing_unit = hub.KerasLayer(bert_preprocessing_url)

In [29]:
text = ['I like the movie', 'Auro is a bad dog']
preprocessed_text = preprocessing_unit(text)


In [31]:
preprocessed_text.keys()

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])

In [34]:
preprocessed_text['input_word_ids']

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[ 101, 1045, 2066, 1996, 3185,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [ 101, 8740, 3217, 2003, 1037, 2919, 3899,  102,    0,    0,    0,
           0,

In [35]:
encoded_text = bert_model(preprocessed_text)

In [36]:
type(encoded_text)

dict

In [37]:
encoded_text.keys()

dict_keys(['pooled_output', 'encoder_outputs', 'default', 'sequence_output'])

In [39]:
encoded_text['pooled_output']

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7997609 , -0.21087182,  0.51771826, ...,  0.39698017,
        -0.5365357 ,  0.82968855],
       [-0.8366005 , -0.36955562, -0.41739225, ..., -0.12472761,
        -0.5702867 ,  0.85754997]], dtype=float32)>

# Build Model

In [52]:
# Bert Model
input_text = tf.keras.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = preprocessing_unit(input_text)
encoded_text = bert_model(preprocessed_text)

# keras
l = tf.keras.layers.Dropout(0.1, name="dropout")(encoded_text['pooled_output'])
out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

model = tf.keras.Model(inputs=[input_text], outputs=[out])


In [53]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_5 (KerasLayer)     {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [54]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
hist = model.fit(df['text'], df['label_num'], epochs=5)

Epoch 1/5
  7/162 [>.............................] - ETA: 35:56 - loss: 0.5985 - accuracy: 0.7098